In [ ]:
# Nvidia Apex - Extension for using Pytorch
# Url - https://github.com/NVIDIA/apex

!cp -rp ../input/nvidiaapex/apex-master/* ./

!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
"""
timm packages
Pytoch Image Models - with updated weights
https://pypi.org/project/timm/
"""

"""
Pycocotools

Dependency for Dectectron2 

COCO (Common Objects in Context) - Image dataset for object segmentation, Recognition

330K Images, > 200K Labels

https://cocodataset.org/#home
"""

In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")

In [ ]:
"""
Ensemble Boxes

https://www.kaggle.com/c/open-images-2019-object-detection/discussion/115086
"""

In [ ]:
from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
import random
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from datetime import datetime
import time
import cv2
import gc
import os
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain,DetBenchEval
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from sklearn.model_selection import StratifiedKFold, train_test_split
from effdet.efficientdet import HeadNet
import effdet.anchors as anchors
from timm.scheduler import CosineLRScheduler
import math
from copy import deepcopy
from apex import amp
import torch.nn as nn

SEED = 1213

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
# Weights used
trained_weight_path = "best_model_path.pth"

In [ ]:
"""
shonenkov Kernel of TTA and Pseudo Labels
"""

In [ ]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = 1024

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [1,3]]
        res_boxes[:, [1,3]] = boxes[:, [2,0]]
        return res_boxes

class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

In [ ]:
"""
Data Augmentation - Modifications to training set
Test Time Augmentation (TTA) - Random modifications(rotations, flipping, translations) to test images. Augmented images. Average results to get more answers

https://www.kaggle.com/andrewkh/test-time-augmentation-tta-worth-it
"""

In [ ]:
"""
itertools -> product -> similar to nested for-loop
"""

In [ ]:
from itertools import product

tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

In [ ]:
def make_tta_predictions(images, nets, score_threshold=0.25):
    all_predictions = []
    images = torch.stack(images).float().cuda()
    for fold_num, net in enumerate(nets):
        with torch.no_grad():
            predictions = []
            for tta_transform in tta_transforms:
                result = []
                det = net(tta_transform.batch_augment(images.clone()), torch.tensor([1]*images.shape[0]).float().cuda())

                for i in range(images.shape[0]):
                    boxes = det[i].detach().cpu().numpy()[:,:4]    
                    scores = det[i].detach().cpu().numpy()[:,4]
                    indexes = np.where(scores > score_threshold)[0]
                    boxes = boxes[indexes]
                    boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                    boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                    boxes = tta_transform.deaugment_boxes(boxes.copy())
                    result.append({
                        'boxes': boxes,
                        'scores': scores[indexes],
                    })
                predictions.append(result)
            all_predictions.append(predictions)
            
    return all_predictions

def run_wbf(all_predictions, image_index, image_size=1024, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    for i in range(len(nets)):
        predictions = all_predictions[i]
        if i == 0:
            boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
            scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
            labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
        else:
            boxes += [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
            scores += [prediction[image_index]['scores'].tolist()  for prediction in predictions]
            labels += [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
            
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels